In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout,TimeDistributed,SimpleRNN
import numpy as np
import pandas as pd
import copy
df = pd.read_csv("../Data/data_8_nodes_yen.csv") #You need to change #directory accordingly
df.head()

,"X(1, 2)","X(1, 3)","X(2, 4)","X(2, 5)","X(3, 4)","X(3, 7)","X(4, 6)","X(5, 6)","X(5, 8)","X(6, 7)",...,"Y4(1, 3)","Y4(2, 4)","Y4(2, 5)","Y4(3, 4)","Y4(3, 7)","Y4(4, 6)","Y4(5, 6)","Y4(5, 8)","Y4(6, 7)","Y4(7, 8)"
0,10,40,30,50,30,20,20,10,90,70,...,0,1,0,0,0,1,0,0,1,1
1,90,50,50,30,30,80,80,30,30,80,...,1,0,0,1,0,1,1,1,0,0
2,10,60,80,50,40,50,30,90,80,30,...,1,0,0,1,0,1,0,0,1,1
3,80,20,40,60,50,80,90,60,80,80,...,1,0,0,1,0,1,1,1,0,0
4,20,70,50,40,20,70,80,80,10,10,...,0,1,0,0,0,1,1,1,0,0


In [2]:
edges = [[1, 2], [1, 3], [2, 4], [2, 5], [3, 4], [3, 7], [4, 6], [5, 6], [5, 8], [6, 7], [7, 8]]

In [3]:
number_edges = 11
number_steps = 1
start_node = 1
end_node = 8

In [4]:
X = df.iloc[:10000,:number_edges].values
y = df.iloc[:10000,number_edges:(number_edges*(number_steps+1))].values

In [5]:
X.shape

(10000, 11)

In [6]:
y.shape

(10000, 11)

In [7]:
from sklearn.model_selection import train_test_split
X1,X2,y_train,y_test = train_test_split(X,y,test_size = 0.1)

In [8]:
X2[1]

array([80, 10, 70, 70, 40, 20, 80, 90, 60, 50, 80], dtype=int64)

In [9]:
y_test[1]

array([0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1], dtype=int64)

In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X1)
X_test= sc.fit_transform(X2)

In [11]:
X_train = np.repeat(X_train[:, np.newaxis, : ], number_steps, axis=1)
X_train.shape

(9000, 1, 11)

In [12]:
X_test = np.repeat(X_test[:, np.newaxis, : ], number_steps, axis=1)
X_test.shape

(1000, 1, 11)

In [13]:
y_train[0]

array([1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0], dtype=int64)

In [14]:
y_train.shape

(9000, 11)

In [15]:
y_train_2 = np.reshape(y_train,(y_train.shape[0], number_steps, number_edges))
y_train_2.shape

(9000, 1, 11)

In [16]:
y_train_2[0]

array([[1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0]], dtype=int64)

In [17]:
y_test_2 = np.reshape(y_test,(y_test.shape[0], number_steps, number_edges))
y_test_2.shape

(1000, 1, 11)

In [18]:
X2[1,:]

array([80, 10, 70, 70, 40, 20, 80, 90, 60, 50, 80], dtype=int64)

In [19]:
import copy
def cost_path(x,y):
    cost = 0
    path = []
    for i in range(number_edges):
        if y[i]==1:
            path.append(edges[i])
            cost+= x[i]
    if(len(path)!=0):
        path_2 = copy.deepcopy(path)
        path_nodes = [start_node]
        if(path[0][0]!=start_node):
            cost = 0
        else:
            path_nodes.append(path[0][1])
            del path_2[0]
            condition = True
            while(len(path_2)!=0 and condition):
                condition = False
                for i in range(len(path_2)):
                    if(path_nodes[-1] in path_2[i]):
                        path_2[i].remove(path_nodes[-1])
                        path_nodes.append(path_2[i][0])
                        path_2.pop(i)
                        condition = True
                        break
        if(path_nodes[-1]!=end_node or len(path_2)!=0):
            cost = 0
    else:
        cost = 0
    return cost

In [20]:
class Network:
    def __init__(self):
        self.model = Sequential()
        self.model.add(SimpleRNN(16, input_shape=(number_steps, number_edges), return_sequences=True))
        self.model.add(TimeDistributed(Dense(number_edges, activation='sigmoid')))
        self.model.compile(loss='BinaryCrossentropy', optimizer='sgd')
        self.fitness = float('inf')
        self.fitness_vector = 0
        self.counter = 0
        self.prob = 0
  
    def set_weights(self, weights):
        self.model.set_weights(weights) 
    
    def get_weights(self):
        return self.model.get_weights()
    
    def architecture(self):
        self.model.summary()
    
    def train(self):
        self.model.fit(X_train, y_train_2, epochs=100, batch_size=64)
    
    def evaluate(self):
        loss = self.model.evaluate(X_train, y_train_2)
        return loss
    
    def get_accuracy(self):
        pred = self.model.predict(X_train)
        accuracy = np.zeros(number_steps+1)
        for i in range(pred.shape[0]):
            condition_1 = True
            for t in range(number_steps):
                output = copy.deepcopy(pred[i][t])
                condition_2 = True
                for j in range(len(output)):
                    output[j] = round(output[j])
                if(cost_path(X1[i],output) != cost_path(X1[i],y_train_2[i][t])):
                    condition_2 = False
                    condition_1 = False
                if condition_2:
                    accuracy[t] += 1
            if condition_1:
                accuracy[number_steps] += 1
        accuracy = np.round(accuracy / pred.shape[0] * 100,2)
        return accuracy[-1]

In [21]:
class ABC:
    def __init__(self, N, Max):
        self.N = N
        self.Max = Max
        self.population = [Network() for i in range(self.N)]
        self.best = Network()
        self.children_population_weights = []
        self.shapes = [a.shape for a in Network().get_weights()]
        self.limit = self.N/2
    
    def Train_generation(self):
        i =  0
        for member in self.population:
            print("Train for member: ",i)
            i+=1
            member.train()
    
    def Train_best(self):
        self.best.train()
    
    def Flatten(self, W):
        flattened = np.concatenate([a.flatten() for a in W])
        return flattened
    
    def Un_Flatten(self, flattened):
        newarray = []
        index = 0
        for shape in self.shapes:
            size = np.product(shape)
            newarray.append(flattened[index : index + size].reshape(shape))
            index += size
        return newarray
    
    def Calculate_fitness(self):
        for i in range(len(self.population)):
            self.population[i].fitness = self.population[i].evaluate()
            
    def EmployeedPhase(self):
        print("Employeed phase:")
        self.children_population_weights.clear()
        counter = [a.counter for a in self.population]
        fitness = [a.fitness for a in self.population]
        for i in range(self.N):
            r = list(range(i)) + list(range(i+1,self.N))
            c = np.random.choice(r)
            coceg_w0 = self.population[c].get_weights()
            coceg = self.Flatten(coceg_w0)
            original_w0 = self.population[i].get_weights()
            original = self.Flatten(original_w0)
            k = np.random.randint(0,int(len(original)/4))
            for ii in range(k):
                d = np.random.randint(len(original))
                fi = np.random.uniform(-1,1)
                original[d] = original[d]+fi*(original[d]-coceg[d])
            origina_w = self.Un_Flatten(original)
            solution = Network()
            solution.set_weights(origina_w)
            solution.fitness = solution.evaluate()
            if(solution.fitness < fitness[i]):
                counter[i] = 0
                fitness[i] = solution.fitness
                self.children_population_weights.append(origina_w)
            else:
                counter[i]+=1
                self.children_population_weights.append(original_w0)
        for i in range(self.N):
            self.population[i].set_weights(self.children_population_weights[i])
            self.population[i].fitness = fitness[i]
            self.population[i].counter = counter[i]
    
    def OnlookedPhase(self):
        print("Onlooked phase:")
        for i in range(self.N):
            self.population[i].fitness_vector = 1/self.population[i].fitness
        summ = 0
        for i in range(self.N):
            summ+=self.population[i].fitness_vector
        for i in range(self.N):
            self.population[i].prob = self.population[i].fitness_vector/summ
        prob=[a.prob for a in self.population]
        for i in range(self.N):
            index_solution = np.random.choice(list(range(self.N)),p = prob)
            r = list(range(index_solution)) + list(range(index_solution+1,self.N))
            c = np.random.choice(r)
            coceg_w0 = self.children_population_weights[c]
            coceg = self.Flatten(coceg_w0)
            original_w0 = self.children_population_weights[index_solution]
            original = self.Flatten(original_w0)
            k = np.random.randint(0,int(len(original)/4))
            for ii in range(k):
                d = np.random.randint(len(original))
                fi = np.random.uniform(-1,1)
                original[d] = original[d]+fi*(original[d]-coceg[d])
            origina_w = self.Un_Flatten(original)
            solution = Network()
            solution.set_weights(origina_w)
            solution.fitness = solution.evaluate()
            if(solution.fitness < self.population[index_solution].fitness):
                self.population[index_solution].counter = 0
                self.population[index_solution].fitness = solution.fitness
                self.population[index_solution].set_weights(origina_w)
            else:
                self.population[index_solution].counter +=1
        
    def ScoutPhase(self):
        print("Scout phase:")
        self.population.sort(key=lambda x: x.fitness)
        n = int(0.6*self.N)
        for i in range(n,self.N):
            if (self.population[i].counter > self.limit):
                old_weight = self.population[0].get_weights()
                for ii in range(len(old_weight)):
                    addition = np.random.uniform(-0.1, 0.1, size=old_weight[ii].shape)
                    old_weight[ii] = old_weight[ii] + addition
                self.population[i].set_weights(old_weight)
                self.population[i].fitness = self.population[i].evaluate()
                self.population[i].counter = 0
    
    def GetBest(self):
        for i in range(self.N):
            print(self.population[i].counter,self.population[i].fitness)
        if (self.population[0].fitness < self.best.fitness):
            self.best.set_weights(self.population[0].get_weights())
            self.best.fitness = self.population[0].fitness

    def Do(self):
        self.Calculate_fitness()
        for i in range(self.Max):
            print("i = ",i)
            self.EmployeedPhase()
            self.OnlookedPhase()
            self.ScoutPhase()
            self.GetBest()
            print("best fitness:",self.best.fitness)

In [22]:
N = 20
Max = 1000

In [23]:
abc = ABC(N, Max)
abc.Do()

282/282 [==============================] - 0s 1ms/step - loss: 0.7148
i =  0
Employeed phase:
282/282 [==============================] - 1s 1ms/step - loss: 0.7185
Onlooked phase:
282/282 [==============================] - 1s 1ms/step - loss: 0.7849
Scout phase:
0 0.705348551273346
2 0.714819073677063
3 0.7313773036003113
1 0.7328911423683167
1 0.734932005405426
1 0.7356081008911133
1 0.7361084818840027
1 0.7377445101737976
3 0.7396480441093445
2 0.7411507964134216
2 0.7420234680175781
1 0.7468769550323486
2 0.751352071762085
1 0.7579179406166077
1 0.759386420249939
2 0.7598426938056946
2 0.7693674564361572
2 0.7695587277412415
2 0.771470308303833
3 0.7848752737045288
best fitness: 0.705348551273346
i =  1
Employeed phase:
282/282 [==============================] - 1s 2ms/step - loss: 0.7837
Onlooked phase:
282/282 [==============================] - 1s 2ms/step - loss: 0.7747
Scout phase:
1 0.705348551273346
2 0.7082411646842957
0 0.7293010354042053
3 0.7328911423683167
0 0.73399299383

282/282 [==============================] - 0s 1ms/step - loss: 0.7783
Onlooked phase:
282/282 [==============================] - 1s 1ms/step - loss: 0.7546
Scout phase:
0 0.7000201940536499
1 0.7075349688529968
1 0.7265045642852783
2 0.7291106581687927
5 0.7328911423683167
0 0.7329639196395874
1 0.7339929938316345
0 0.7341691851615906
0 0.737143874168396
0 0.7380218505859375
1 0.7407594919204712
1 0.7435696125030518
0 0.7478410601615906
2 0.7531991600990295
2 0.7576472163200378
4 0.759386420249939
0 0.7691341638565063
4 0.7695587277412415
2 0.7711065411567688
0 0.7782503962516785
best fitness: 0.7000201940536499
i =  3
Employeed phase:
282/282 [==============================] - 1s 2ms/step - loss: 0.7741
Onlooked phase:
282/282 [==============================] - 0s 1ms/step - loss: 0.7735
Scout phase:
2 0.7000201940536499
2 0.7075349688529968
1 0.7241869568824768
0 0.726621150970459
0 0.7266676425933838
3 0.7291106581687927
0 0.7326723337173462
0 0.7327132821083069
1 0.7341691851615906

282/282 [==============================] - 1s 1ms/step - loss: 0.7764
Scout phase:
0 0.698437511920929
1 0.7029673457145691
0 0.7131851315498352
3 0.7241869568824768
2 0.7266676425933838
6 0.7291106581687927
1 0.7315523028373718
1 0.7326723337173462
2 0.7327132821083069
1 0.7332322597503662
1 0.7364850640296936
1 0.738699734210968
1 0.7391934394836426
0 0.7399668097496033
0 0.7489486932754517
1 0.7527793049812317
4 0.753743052482605
1 0.7656856775283813
2 0.7665507197380066
2 0.7682371735572815
best fitness: 0.698437511920929
i =  5
Employeed phase:
282/282 [==============================] - 1s 2ms/step - loss: 0.7708
Onlooked phase:
282/282 [==============================] - 1s 1ms/step - loss: 0.7249
Scout phase:
0 0.6939778923988342
2 0.6963651180267334
1 0.7131851315498352
0 0.7195380330085754
1 0.7228766679763794
0 0.7274623513221741
0 0.7276530265808105
4 0.7326723337173462
3 0.7327132821083069
3 0.7332322597503662
3 0.7364850640296936
5 0.738699734210968
1 0.7389820218086243
3 0

282/282 [==============================] - 1s 2ms/step - loss: 0.7984
Onlooked phase:
282/282 [==============================] - 0s 1ms/step - loss: 0.7408
Scout phase:
0 0.6898676156997681
5 0.6963651180267334
3 0.7131851315498352
3 0.7179087996482849
2 0.7202517986297607
4 0.7274623513221741
4 0.7276530265808105
0 0.7322860360145569
7 0.7327132821083069
9 0.7332322597503662
1 0.7340407371520996
6 0.7364850640296936
0 0.7383723258972168
5 0.7391934394836426
5 0.7399668097496033
5 0.7403154373168945
0 0.741335391998291
1 0.7560054063796997
0 0.7605084776878357
5 0.7665507197380066
best fitness: 0.6898676156997681
i =  8
Employeed phase:
282/282 [==============================] - 1s 1ms/step - loss: 0.7837
Onlooked phase:
282/282 [==============================] - 1s 1ms/step - loss: 0.7635
Scout phase:
0 0.6887698769569397
1 0.6898676156997681
1 0.7124526500701904
6 0.7179087996482849
3 0.7202517986297607
0 0.7239122986793518
1 0.725254476070404
5 0.7274623513221741
10 0.73271328210830

282/282 [==============================] - 1s 1ms/step - loss: 0.7343
Scout phase:
0 0.6873860955238342
1 0.6887698769569397
0 0.6890146732330322
9 0.7179087996482849
7 0.7202517986297607
2 0.7239122986793518
4 0.725254476070404
7 0.7274623513221741
1 0.7295915484428406
0 0.7329732775688171
12 0.7332322597503662
1 0.733390748500824
0 0.7342726588249207
0 0.7361169457435608
1 0.7384160757064819
8 0.7399668097496033
8 0.7403154373168945
5 0.7560054063796997
7 0.7605084776878357
7 0.7665507197380066
best fitness: 0.6873860955238342
i =  10
Employeed phase:
282/282 [==============================] - 1s 1ms/step - loss: 0.7663
Onlooked phase:
282/282 [==============================] - 1s 1ms/step - loss: 0.7422
Scout phase:
282/282 [==============================] - 0s 2ms/step - loss: 0.7069
1 0.6873860955238342
2 0.6887698769569397
2 0.6890146732330322
1 0.7017703056335449
2 0.7135361433029175
1 0.7211161255836487
0 0.7213663458824158
4 0.7239122986793518
8 0.7274623513221741
0 0.72875350

...

282/282 [==============================] - 1s 1ms/step - loss: 0.1143
Scout phase:
14 0.11385857313871384
14 0.11388488858938217
6 0.11407680809497833
3 0.11420231312513351
0 0.11476036161184311
0 0.11477401852607727
2 0.1147843524813652
4 0.11562836170196533
1 0.11647830903530121
8 0.11649724841117859
1 0.11663013696670532
2 0.11681198328733444
2 0.11686954647302628
9 0.11719442903995514
1 0.11772380769252777
4 0.11786090582609177
0 0.11846655607223511
1 0.11900386214256287
1 0.11931066960096359
2 0.11969886720180511
best fitness: 0.11385857313871384
i =  526
Employeed phase:
282/282 [==============================] - 1s 1ms/step - loss: 0.1198
Onlooked phase:
282/282 [==============================] - 1s 1ms/step - loss: 0.1190
Scout phase:
1 0.11378316581249237
18 0.11388488858938217
7 0.11407680809497833
2 0.11410137265920639
0 0.11451927572488785
1 0.11476036161184311
4 0.1147843524813652
1 0.11558271944522858
2 0.11647830903530121
9 0.11649724841117859
0 0.11654014885425568
0 0.1

282/282 [==============================] - 1s 1ms/step - loss: 0.1198
Onlooked phase:
282/282 [==============================] - 1s 1ms/step - loss: 0.1167
Scout phase:
3 0.11378316581249237
22 0.11388488858938217
9 0.11407680809497833
7 0.11410137265920639
2 0.11438482999801636
5 0.11451927572488785
3 0.11475875973701477
1 0.11523182690143585
1 0.11535051465034485
1 0.1164461150765419
8 0.11647830903530121
0 0.11677294969558716
1 0.11686089634895325
2 0.11703092604875565
2 0.11730148643255234
0 0.11750370264053345
4 0.1183488667011261
6 0.11846655607223511
6 0.11931066960096359
7 0.11969886720180511
best fitness: 0.11378316581249237
i =  529
Employeed phase:
282/282 [==============================] - 1s 1ms/step - loss: 0.1197
Onlooked phase:
282/282 [==============================] - 1s 1ms/step - loss: 0.1174
Scout phase:
0 0.11371369659900665
4 0.11378316581249237
1 0.11379525065422058
8 0.11410137265920639
3 0.11438482999801636
1 0.11450199037790298
4 0.11475875973701477
0 0.11517

282/282 [==============================] - 1s 1ms/step - loss: 0.1152
Scout phase:
282/282 [==============================] - 0s 1ms/step - loss: 0.1203
1 0.11371369659900665
5 0.11378316581249237
3 0.11379525065422058
0 0.1139882281422615
4 0.11438482999801636
4 0.11450199037790298
2 0.11458444595336914
2 0.11517666280269623
5 0.11535051465034485
0 0.1161976307630539
0 0.11627193540334702
13 0.11647830903530121
8 0.11686089634895325
4 0.11703092604875565
2 0.11716552823781967
0 0.11721229553222656
0 0.1179889440536499
2 0.11806639283895493
3 0.11915011703968048
0 0.12033821642398834
best fitness: 0.11371369659900665
i =  531
Employeed phase:
282/282 [==============================] - 1s 1ms/step - loss: 0.1204
Onlooked phase:
282/282 [==============================] - 1s 1ms/step - loss: 0.1178
Scout phase:
2 0.11371369659900665
8 0.11378316581249237
5 0.11379525065422058
4 0.1139882281422615
1 0.11436067521572113
6 0.11450199037790298
5 0.11458444595336914
4 0.11517666280269623
0 0.1

282/282 [==============================] - 1s 1ms/step - loss: 0.1205
Onlooked phase:
282/282 [==============================] - 1s 1ms/step - loss: 0.1154
Scout phase:
1 0.1136658638715744
0 0.11373381316661835
7 0.11379525065422058
0 0.11398783326148987
4 0.11436067521572113
10 0.11450199037790298
8 0.11458444595336914
2 0.11496037989854813
6 0.11517666280269623
1 0.11601302027702332
7 0.11601563543081284
6 0.1161976307630539
1 0.11630592495203018
1 0.1166890487074852
0 0.11677978187799454
2 0.11712057888507843
2 0.11735591292381287
0 0.11775217205286026
2 0.11785919964313507
0 0.11945690959692001
best fitness: 0.1136658638715744
i =  534
Employeed phase:
282/282 [==============================] - 1s 1ms/step - loss: 0.1199
Onlooked phase:
165/282 [================>.............] - ETA: 0s - loss: 0.1139

KeyboardInterrupt: 

In [24]:
abc.best.get_accuracy()

282/282 [==============================] - 1s 1ms/step


87.68

In [25]:
abc.best.evaluate()

282/282 [==============================] - 1s 1ms/step - loss: 0.1137


0.1136658638715744